**<font color = black size = 5>All of the component in the data collection<font>**

In [1]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json

import glob
import datetime
import numpy as np
import cv2
import time

 **<font size = 5>Make Directory for Data</font>**

In [ ]:
DATASET_DIR = 'dataset_merge_v3'
block_dir =os.path.join(DATASET_DIR, 'block')
free_dir = os.path.join(DATASET_DIR, 'free')


try:
    os.makedirs(block_dir)
    os.makedirs(free_dir)
except FileExistsError:
    print('Directories not created because they already exist')

**<font size = 5>Get Camera Image And Button</font>**

In [ ]:
camera = Camera.instance(width=224, height=224,fps=1)

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

leftBottom = widgets.FloatSlider(min =-2.0, max=2.0, step=0.01, description='leftButtom')
leftTop = widgets.FloatSlider(min =-2.0, max=2.0, step=0.01, description='leftTop')
rightBottom = widgets.FloatSlider(min =-2.0, max=2.0, step=0.01, description='rightButtom')
rightTop = widgets.FloatSlider(min =-2.0, max=2.0, step=0.01, description='rightTop')                               
                                
def display_xy(camera_image):
    image = np.copy(camera_image)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # ret, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
    image = cv2.GaussianBlur(image, (5,5), 0)
    # image = cv2.GaussianBlur(image, (5,5), 0)
    image = cv2.Canny(image, 130, 150)
     
#     rho = 1
#     theta = np.pi / 180
#     threshold = 1
#     min_line_length = 10
#     max_line_gap = 1
    
#     line_image = np.copy(image) * 0
#     lines = cv2.HoughLinesP(image, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)
    
#     for x1, y1, x2, y2 in lines[0]:
#         cv2.line(line_image, (x1, y1), (x2, y2), (255, 255, 255), 10)
    
#     image = cv2.addWeighted(image, 0.8, line_image, 1, 0) 
    
    LB = leftBottom.value
    LT = leftTop.value
    RB = rightBottom.value
    RT = rightTop.value

    LB = int (LB * 224 / 2 + 112) 
    LT = int (LT * 224 / 2 + 112)                            
    RB = int (RB * 224 / 2 + 112)                            
    RT = int (RT * 224 / 2 + 112)     
    CB = int((LB + RB) / 2)
    CT = int((LT + RT) / 2)
    CC = int((CB + CT) / 2)
                               
    image = cv2.line(image, (LB, 224), (LT, 0), (255, 0 ,0), 2)
    image = cv2.line(image, (RB, 224), (RT, 0), (255, 0, 0), 2)
    image = cv2.line(image, (CB, 224), (CT, 0), (255, 0, 0), 2)
    image = cv2.circle(image, (CC, 112),8, (255, 0, 0), 2)   
    
    jpeg_image = bgr8_to_jpeg(image)
    # print(x, y)
    return jpeg_image

# time.sleep(10)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

**<font size = 5 color = blue> Start to Collect the Data(Including the number of picture)</font>**

In [4]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='primary', layout=button_layout)
block_button = widgets.Button(description='add block', button_style='warning', layout=button_layout)
free_count = widgets.IntText(value=len(os.listdir(free_dir)),layout=button_layout)
block_count = widgets.IntText(value=len(os.listdir(block_dir)),layout=button_layout)

In [5]:
def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 224 / 2 + 224 / 2, y * 224 / 2 + 224 / 2, uuid1())

def save_snapshot(directory):
    uuid = xy_uuid((leftBottom.value + leftTop.value + rightBottom.value + rightTop.value) / 4, 0)
    image_path = os.path.join(directory, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
        
def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value=len(os.listdir(free_dir))
    # print('free picture(s) ' + len(os.listdir(free_dir)))
    
def save_block():
    global block_dir, block_count
    save_snapshot(block_dir)
    block_count.value=len(os.listdir(block_dir))
    # print('block picture(s) ' + len(os.listdir(block_dir)))

free_button.on_click(lambda x:save_free())
block_button.on_click(lambda x:save_block()) 

In [6]:
# display(widgets.HBox([image_widget, target_widget]))
display(widgets.HBox([image_widget, target_widget]))
# display(target_widget)
display(widgets.HBox([widgets.VBox([leftTop, leftBottom]), widgets.VBox([rightTop, rightBottom])]))

display(widgets.VBox([widgets.HBox([free_count, block_count]), widgets.HBox([free_button, block_button])]))

# display(widgets.HBox([free_button, block_button]))


# display(free_count)
# display(free_button)
# display(block_button)

In [ ]:
print('block picture(s) ' + str(len(os.listdir(block_dir))))
print('free picture(s) ' + str(len(os.listdir(free_dir))))

In [7]:
camera.stop()

**<font size = 5>Use Zip to low store total directory</font>**

In [2]:
DATASET_DIR = 'dataset_merge_v2'


!zip -r -q {DATASET_DIR}.zip {DATASET_DIR}

In [ ]:
import cv2
import os
import numpy as np

ORIGINAL_DIR = 'dataset_xy'
DATASET_DIR = 'dataset_xy_reduce'
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print("The directory is exist!!")

    
# img = cv2.imread("dataset_xy/xy_033_171_45308aac-bdf7-11ec-be0d-16f63a1aa8c9.jpg", 0)
# img = cv2.medianBlur(img, 5)
# img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, -1)
# cv2.imwrite(DATASET_DIR + "/"  + "xy_033_171_45308aac-bdf7-11ec-be0d-16f63a1aa8c9" + ".jpg", img)    

kernel = np.ones((3,3), np.uint8)
for image in os.listdir(ORIGINAL_DIR):
    # pass
    img = cv2.imread(ORIGINAL_DIR + "/" + image, 0)
    img = cv2.medianBlur(img, 5)
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, -1)
    
    img = cv2.erode(img, kernel, iterations = 1)
    img = cv2.dilate(img, kernel, iterations = 1)
    
    cv2.imwrite(DATASET_DIR + "/" + image, img)
  